# Capstone Project - The Battle of the Neighborhoods, Turin
### Applied Data Science Capstone by IBM/Coursera

## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)



## Introduction: Business Problem <a name="introduction"></a>

In this project we will try to find an optimal location for a restaurant. Specifically, this report will be targeted to stakeholders interested in opening a **vegetarian/vegan restaurant** in **Turin**, Italy.

Since there are lots of restaurants in Turin (particularly in the center) we will try to detect **locations that are not already crowded with restaurants**. We are also particularly interested in **areas with no vegetarian or vegan restaurants in vicinity**. We would also prefer locations **as close to city center as possible**, assuming that first two conditions are met.

We will use our data science powers to generate a few most promising neighborhoods based on this criteria. Advantages of each area will then be clearly expressed so that best possible final location can be chosen by stakeholders.

## Data <a name="data"></a>

Based on definition of our problem, factors that will influence our decission are:
* number of existing restaurants in the neighborhood (any type of restaurant)
* number of and distance to vegetarian or vegan restaurants in the neighborhood, if any
* distance of neighborhood from city center

We decided to use regularly spaced grid of locations, centered around city center, to define our neighborhoods.

Following data sources will be needed to extract/generate the required information:
* centers of candidate areas will be generated algorithmically and approximate addresses of centers of those areas will be obtained using **Open Street Map API reverse geocoding**
* number of restaurants and their type and location in every neighborhood will be obtained using **Foursquare API**
* coordinate of Turin center will be obtained using **Open Street Map API geocoding** of well known Turin location (Piazza Castello)

### Neighborhood Candidates

Let's create latitude & longitude coordinates for centroids of our candidate neighborhoods. We will create a grid of cells covering our area of interest which is aprox. 3x3 killometers centered around Turin city center.

Let's first find the latitude & longitude of Turin city center, using specific, well known address (Piazza Castello) and Open Street Map geocoding API.

In [1]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [2]:
import sys
!{sys.executable} -m pip install geocoder
import geocoder # import geocoder

# initialize your variable to None
lat_lng_coords = None

In [3]:
import requests

while(lat_lng_coords is None):
  g = geocoder.osm('{}, Piazza Castello, Torino, Italia')
  lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]

address = 'Piazza Castello, Torino, Italia'

print('Coordinate of {}: latitude: {}, longitude: {}'.format(address, latitude, longitude))

Coordinate of Piazza Castello, Torino, Italia: latitude: 45.0710035, longitude: 7.6869227


Now let's create a grid of area candidates, equaly spaced, centered around city center and within ~3km from Piazza Castello. Our neighborhoods will be defined as circular areas with a radius of 300 meters, so our neighborhood centers will be 600 meters apart.

To accurately calculate distances we need to create our grid of locations in Cartesian 2D coordinate system which allows us to calculate distances in meters (not in latitude/longitude degrees). Then we'll project those coordinates back to latitude/longitude degrees to be shown on Folium map. So let's create functions to convert between WGS84 spherical coordinate system (latitude/longitude degrees) and UTM Cartesian coordinate system (X/Y coordinates in  meters).

In [4]:
!pip install shapely
import shapely.geometry

!pip install pyproj
import pyproj

In [5]:
import math

def lonlat_to_xy(lon, lat):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=33, datum='WGS84')
    xy = pyproj.transform(proj_latlon, proj_xy, lon, lat)
    return xy[0], xy[1]

def xy_to_lonlat(x, y):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=33, datum='WGS84')
    lonlat = pyproj.transform(proj_xy, proj_latlon, x, y)
    return lonlat[0], lonlat[1]

def calc_xy_distance(x1, y1, x2, y2):
    dx = x2 - x1
    dy = y2 - y1
    return math.sqrt(dx*dx + dy*dy)

print('Coordinate transformation check')
print('-------------------------------')
print('Turin center longitude={}, latitude={}'.format(longitude, latitude))
x, y = lonlat_to_xy(longitude, latitude)
print('Turin center UTM X={}, Y={}'.format(x, y))
lo, la = xy_to_lonlat(x, y)
print('Turin center longitude={}, latitude={}'.format(lo, la))

Coordinate transformation check
-------------------------------
Turin center longitude=7.6869227, latitude=45.0710035
Turin center UTM X=-75664.88757318573, Y=5016919.14830222
Turin center longitude=7.686922699999999, latitude=45.0710035


Let's create a **hexagonal grid of cells**: we offset every other row, and adjust vertical row spacing so that **every cell center is equally distant from all it's neighbors**.

In [6]:
turin_center_x, turin_center_y = lonlat_to_xy(longitude, latitude) # City center in Cartesian coordinates

k = math.sqrt(3) / 2 # Vertical offset for hexagonal grid cells
x_min = turin_center_x - 3000
x_step = 600
y_min = turin_center_y - 3000 - (int(21/k)*k*600 - 6000)/2
y_step = 600 * k 

latitudes = []
longitudes = []
distances_from_center = []
xs = []
ys = []
for i in range(0, int(21/k)):
    y = y_min + i * y_step
    x_offset = 300 if i%2==0 else 0
    for j in range(0, 21):
        x = x_min + j * x_step + x_offset
        distance_from_center = calc_xy_distance(turin_center_x, turin_center_y, x, y)
        if (distance_from_center <= 3001):
            lon, lat = xy_to_lonlat(x, y)
            latitudes.append(lat)
            longitudes.append(lon)
            distances_from_center.append(distance_from_center)
            xs.append(x)
            ys.append(y)

print(len(latitudes), 'candidate neighborhood centers generated.')

92 candidate neighborhood centers generated.


Let's visualize the data we have so far: city center location and candidate neighborhood centers:

In [7]:
!pip install folium

import folium

In [8]:
map_turin = folium.Map(location=lat_lng_coords, zoom_start=13)
folium.Marker(lat_lng_coords, popup='Piazza Castello').add_to(map_turin)
for lat, lon in zip(latitudes, longitudes):
    #folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_berlin) 
    folium.Circle([lat, lon], radius=300, color='blue', fill=False).add_to(map_turin)
    #folium.Marker([lat, lon]).add_to(map_berlin)
map_turin

OK, we now have the coordinates of centers of neighborhoods/areas to be evaluated, equally spaced (distance from every point to it's neighbors is exactly the same) and within ~3km from Piazza Castello. 

Let's now use Open Street Map API to get approximate addresses of those locations.

In [9]:

def get_address(latitude, longitude):
    try:
        reverse = geocoder.osm([latitude,longitude], method = 'reverse')
        return reverse.address
    except:
        return None

addr = get_address(latitude, longitude)
print('Reverse geocoding check')
print('-----------------------')
print('Address of [{}, {}] is: {}'.format(latitude, longitude, addr))

Reverse geocoding check
-----------------------
Address of [45.0710035, 7.6869227] is: Castello Capolinea, Piazza Castello, Borgo Nuovo, Centro, Torino, Provincia di Torino, Piemonte, 10123, Italia


In [10]:
print('Obtaining location addresses: ', end='')
addresses = []
for lat, lon in zip(latitudes, longitudes):
    address = get_address(lat, lon)
    if address is None:
        address = 'NO ADDRESS'
    address = address.replace(', Italia', '') # We don't need country part of address
    address = address.replace(', Torino, Provincia di Torino, Piemonte', '') # We don't need city and state part of address
    addresses.append(address)
    print(' .', end='')
print(' done.')

Obtaining location addresses:  . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . done.


In [11]:
addresses[50:70] #some examples to see if it worked

['20 scala B, Via Oslavia, Vanchiglietta, 10153',
 'CSEN Centro Sportivo Educativo Nazionale - Comitato Provinciale di Torino, 19, Corso Francesco Ferrucci, Cit Turin, 10128',
 '15/E, Via Giuseppe Grassi, Cit Turin, 10138',
 '16 bis scala A, Via Filippo Juvarra, Quadrilatero Romano, Centro, 10122',
 'San Giovanni Battista, Via San Domenico, Quadrilatero Romano, Centro, 10140',
 '107, Corso Regina Margherita, Centro, 10122',
 '37/I, Via Pisa, Rossini, Aurora, 10153',
 '21/B, Via Carlo Michele Buscalioni, Rossini, Aurora, 10153',
 'Via Poliziano, Rossini, Circoscrizione 7, 10153',
 '32/B, Via Giulio Carcano, Vanchiglietta, 10153',
 '26, Via Digione, Martinetto, Campidoglio, 10143',
 '12/D, Via Stefano Clemente, Basso San Donato, San Donato, 10143',
 'Istituto Sacra Famiglia, Via San Donato, San Donato, 10140',
 '147/A, Corso Regina Margherita, Centro, 10122',
 '28, Via Vittorio Andreis, Borgo Dora, Aurora, 10152',
 '5 scala B, Via Alessandria, Borgo Dora, Aurora, 10152',
 '52, Via Perugi

Looking good. Let's now place all this into a Pandas dataframe.

In [12]:
import pandas as pd

df_locations = pd.DataFrame({'Address': addresses,
                             'Latitude': latitudes,
                             'Longitude': longitudes,
                             'X': xs,
                             'Y': ys,
                             'Distance from center': distances_from_center})

df_locations.head(10)

,Address,Latitude,Longitude,X,Y,Distance from center
0,"64 scala B, Corso Dante Alighieri, San Salvari...",45.046833,7.674782,-76864.887573,5.014321e+06,2861.817604
1,"4/A, Via Carlo Marenco, San Salvario, 10126",45.047320,7.682338,-76264.887573,5.014321e+06,2666.458325
2,"6 int. 1, Strada Antica di San Vito, Crimea, C...",45.047807,7.689895,-75664.887573,5.014321e+06,2598.076211
3,"19 int. 7, Strada di Revigliasco, Crimea, Circ...",45.048293,7.697452,-75064.887573,5.014321e+06,2666.458325
4,"11 int. 5, Strada del Ponte Verde, Crimea, Cir...",45.048778,7.705010,-74464.887573,5.014321e+06,2861.817604
5,"131, Corso Re Umberto, Crocetta, 10128",45.050740,7.662851,-77764.887573,5.014841e+06,2954.657341
6,"ex Ghia, Via Agostino da Montefeltro, San Seco...",45.051228,7.670408,-77164.887573,5.014841e+06,2563.201124
7,"17/I, Corso Raffaello, San Salvario, 10125",45.051716,7.677965,-76564.887573,5.014841e+06,2264.950331
8,"Sport Club - Villa Glicini, Viale Diego Balsam...",45.052203,7.685522,-75964.887573,5.014841e+06,2100.000000
9,"45, Viale Giacomo Curreno, Crimea, Circoscrizi...",45.052689,7.693080,-75364.887573,5.014841e+06,2100.000000


...and let's now save/persist this data into local file.

In [13]:
df_locations.to_pickle('./turin_locations.pkl')

### Foursquare
Now that we have our location candidates, let's use Foursquare API to get info on restaurants in each neighborhood.

We're interested in venues in 'food' category, but only those that are vegan/vegetarian restaurants - coffe shops, pizza places, bakeries etc. are not direct competitors so we don't care about those. So we will include in out list only venues that have 'restaurant' in category name.

In [14]:
#Foursquare credential

CLIENT_ID = 'MAZ3ZS4MDA4ONSCIAJC1VT5FQXOITDDE2FL5SFLD0MYCN2AG' # your Foursquare ID
CLIENT_SECRET = 'A5N0GPLMWF3OE2PVQ5SROJQ4N5RUAZMWJN1PKWSMIE3R5ZW2' # your Foursquare Secret
version = '20180604'

In [15]:
# Category IDs corresponding to vegetarian/vegan restaurants were taken from Foursquare web site (https://developer.foursquare.com/docs/resources/categories):

food_category = '4d4b7105d754a06374d81259' # 'Root' category for all food-related venues

veg_restaurant_categories = ['4bf58dd8d48988d1d3941735']

def is_restaurant(categories, specific_filter=None):
    restaurant_words = ['restaurant', 'diner', 'taverna', 'steakhouse']
    restaurant = False
    specific = False
    for c in categories:
        category_name = c[0].lower()
        category_id = c[1]
        for r in restaurant_words:
            if r in category_name:
                restaurant = True
        if 'fast food' in category_name:
            restaurant = False
        if not(specific_filter is None) and (category_id in specific_filter):
            specific = True
            restaurant = True
    return restaurant, specific

def get_categories(categories):
    return [(cat['name'], cat['id']) for cat in categories]

def format_address(location):
    address = ', '.join(location['formattedAddress'])
    address = address.replace(', Italia', '')
    address = address.replace(', Italy', '')
    address = address.replace(', Torino, Provincia di Torino, Piemonte', '')
    return address

def get_venues_near_location(lat, lon, category, client_id, client_secret, radius=500, limit=100):
    version = '20180724'
    url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&categoryId={}&radius={}&limit={}'.format(
        client_id, client_secret, version, lat, lon, category, radius, limit)
    try:
        results = requests.get(url).json()['response']['groups'][0]['items']
        venues = [(item['venue']['id'],
                   item['venue']['name'],
                   get_categories(item['venue']['categories']),
                   (item['venue']['location']['lat'], item['venue']['location']['lng']),
                   format_address(item['venue']['location']),
                   item['venue']['location']['distance']) for item in results]        
    except:
        venues = []
    return venues

In [16]:
# Let's now go over our neighborhood locations and get nearby restaurants; we'll also maintain a dictionary of all found restaurants and all found vegetarian and vegan restaurants

import pickle

def get_restaurants(lats, lons):
    restaurants = {}
    veg_restaurants = {}
    location_restaurants = []

    print('Obtaining venues around candidate locations:', end='')
    for lat, lon in zip(lats, lons):
        # Using radius=350 to meke sure we have overlaps/full coverage so we don't miss any restaurant (we're using dictionaries to remove any duplicates resulting from area overlaps)
        venues = get_venues_near_location(lat, lon, food_category, CLIENT_ID, CLIENT_SECRET, radius=350, limit=100)
        area_restaurants = []
        for venue in venues:
            venue_id = venue[0]
            venue_name = venue[1]
            venue_categories = venue[2]
            venue_latlon = venue[3]
            venue_address = venue[4]
            venue_distance = venue[5]
            is_res, is_veg = is_restaurant(venue_categories, specific_filter=veg_restaurant_categories)
            if is_res:
                x, y = lonlat_to_xy(venue_latlon[1], venue_latlon[0])
                restaurant = (venue_id, venue_name, venue_latlon[0], venue_latlon[1], venue_address, venue_distance, is_veg, x, y)
                if venue_distance<=300:
                    area_restaurants.append(restaurant)
                restaurants[venue_id] = restaurant
                if is_veg:
                    veg_restaurants[venue_id] = restaurant
        location_restaurants.append(area_restaurants)
        print(' .', end='')
    print(' done.')
    return restaurants, veg_restaurants, location_restaurants

# Try to load from local file system in case we did this before
restaurants = {}
veg_restaurants = {}
location_restaurants = []
loaded = False
try:
    with open('restaurants.pkl', 'rb') as f:
        restaurants = pickle.load(f)
    with open('veg_restaurants.pkl', 'rb') as f:
        veg_restaurants = pickle.load(f)
    with open('location_restaurants.pkl', 'rb') as f:
        location_restaurants = pickle.load(f)
    print('Restaurant data loaded.')
    loaded = True
except:
    pass

# If load failed use the Foursquare API to get the data
if not loaded:
    restaurants, veg_restaurants, location_restaurants = get_restaurants(latitudes, longitudes)
    
    # Let's persists this in local file system
    with open('restaurants.pkl', 'wb') as f:
        pickle.dump(restaurants, f)
    with open('veg_restaurants.pkl', 'wb') as f:
        pickle.dump(veg_restaurants, f)
    with open('location_restaurants.pkl', 'wb') as f:
        pickle.dump(location_restaurants, f)
        

Restaurant data loaded.


In [17]:
import numpy as np

print('Total number of restaurants:', len(restaurants))
print('Total number of vegetarian/vegan restaurants:', len(veg_restaurants))
print('Percentage of vegetarian/vegan restaurants: {:.2f}%'.format(len(veg_restaurants) / len(restaurants) * 100))
print('Average number of restaurants in neighborhood:', np.array([len(r) for r in location_restaurants]).mean())

Total number of restaurants: 420
Total number of vegetarian/vegan restaurants: 8
Percentage of vegetarian/vegan restaurants: 1.90%
Average number of restaurants in neighborhood: 4.119565217391305


In [18]:
print('List of all restaurants')
print('-----------------------')
for r in list(restaurants.values())[:10]:
    print(r)
print('...')
print('Total:', len(restaurants))

List of all restaurants
-----------------------
('4c54670cf5f3d13a719418fa', 'Tientsin Ristorante Cinese', 45.0471497357592, 7.674320682634583, 'Corso Dante 62, 10126 Torino Piemonte', 50, False, -76897.98372273077, 5014359.5782210585)
('4db7fb6004376df99769c9bd', "Ristorante L'Birichin", 45.04855, 7.67657, 'Via V. Monti, 16, 10126 Torino Piemonte', 237, False, -76706.70674569858, 5014499.01281492)
('4c3e3f98d13aef3bc9889339', 'Pasta e Basta', 45.045330276812564, 7.674048143782729, 'Via Madama Cristina 116, 10126 Torino Piemonte', 176, False, -76937.83469036478, 5014159.417203151)
('517c1e7ee4b0eef57509e68a', 'Star Instanbul Kebab', 45.04714763693052, 7.673349380493164, 'Piazza De Amicis, 10134 Torino Piemonte', 117, False, -76974.4930086406, 5014366.304569463)
('4de34b41ae60e7f3ac043472', "Sapori d'Italia", 45.04758510119103, 7.672649393572578, 'Via nizza 51/A, 10126 Torino Piemonte', 187, False, -77025.19321173767, 5014419.916131234)
('59d7c159446ea63601d3b160', 'Japs', 45.047539, 7.

In [19]:
print('List of Vegetarian/Vegan restaurants')
print('---------------------------')
for r in list(veg_restaurants.values())[:10]:
    print(r)
print('...')
print('Total:', len(veg_restaurants))

List of Vegetarian/Vegan restaurants
---------------------------
('57192776498e2192bfa6e8df', 'Gorilla', 45.05947026271252, 7.683088383826403, 'Via Bernardino Galliari 20, 10125 Torino Piemonte', 244, True, -76083.16279835033, 5015665.417608366)
('545f7dad498e55298fea1199', 'Selezione Naturale', 45.06228231824135, 7.688324286870411, 'Via Andrea Provana 3/D (via dei Mille), 10123 Torino Piemonte', 326, True, -75642.563591226, 5015940.338702487)
('558e85cf498e512b2c87edd2', 'Coox', 45.065540638194555, 7.6937919087622815, 'Piazza Vittorio Veneto 6 (Via Principe Amedeo), Torino Piemonte', 177, True, -75179.26289525896, 5016263.204104551)
('584a995d16f00008d5465676', 'Flower Burger', 45.07122871548427, 7.6778296356435565, 'Via Antonio Bertola 29/C, 10121', 206, True, -76378.36945705349, 5017009.241823803)
('5317596a498effeacd1e8fd3', "Verdi d'Indivia", 45.07127130998359, 7.682534169161199, 'Via Barbaroux 5, 10122 Torino Piemonte', 74, True, -76007.62478940608, 5016980.294622011)
('4df745974

In [20]:
print('Restaurants around location')
print('---------------------------')
for i in range(20, 30):
    rs = location_restaurants[i][:8]
    names = ', '.join([r[1] for r in rs])
    print('Restaurants around location {}: {}'.format(i+1, names))

Restaurants around location
---------------------------
Restaurants around location 21: 
Restaurants around location 22: 
Restaurants around location 23: Spazio7, Ristorante Cinese Grande Oriente
Restaurants around location 24: Japs!crocetta, Ristorante Nuova Crocetta, Shri Ganesh, Mar Rosso Kebab
Restaurants around location 25: Ristorante Al Cacimperio
Restaurants around location 26: Taverna delle Rose, La Locanda del Sorriso, Ristorante Giovanni, Ristorante Marcello, Greek Food Lab, Da Aydin Pizza Kebap Bar (Da Aydın), Trattoria Da Felice, Ristorante Urbani
Restaurants around location 27: El Beso, La Piadineria, Contesto Alimentare, Frató restaurant, Tokyo Zen, Osteria degli Ottoni, Mikien Sushi Bar, Gorilla
Restaurants around location 28: Oinos Restaurant
Restaurants around location 29: Ristorante Monferrato, Petit Jardin, La Cantinella, Anema e Core
Restaurants around location 30: 


Let's now see all the collected restaurants in our area of interest on map, and let's also show vegetarian and vegan restaurants in different color.

In [21]:
map_turin = folium.Map(location=lat_lng_coords, zoom_start=13)
folium.Marker(lat_lng_coords, popup='Piazza Castello').add_to(map_turin)
for res in restaurants.values():
    lat = res[2]; lon = res[3]
    is_veg = res[6]
    color = 'red' if is_veg else 'blue'
    folium.CircleMarker([lat, lon], radius=3, color=color, fill=True, fill_color=color, fill_opacity=1).add_to(map_turin)
map_turin

Looking good. So now we have all the restaurants in area within few kilometers from Piazza Castello, and we know which ones are vegetarian or vegan restaurants! We also know which restaurants exactly are in vicinity of every neighborhood candidate center.

This concludes the data gathering phase - we're now ready to use this data for analysis to produce the report on optimal locations for a new veg restaurant!

## Methodology <a name="methodology"></a>

In this project we will direct our efforts on detecting areas of Turin that have low restaurant density, particularly those with low number of vegetarian and vegan restaurants. We will limit our analysis to area ~3km around city center.

In first step we have collected the required **data: location and type (category) of every restaurant within 3km from Turin center** (Piazza Castello). We have also **identified vegetarian and vegan restaurants** (according to Foursquare categorization).

Second step in our analysis will be calculation and exploration of '**restaurant density**' across different areas of Turin - we will use **heatmaps** to identify a few promising areas close to center with low number of restaurants in general (*and* no vegetarian/vegan restaurants in vicinity) and focus our attention on those areas.

In third and final step we will focus on most promising areas and within those create **clusters of locations that meet some basic requirements** established in discussion with stakeholders: we will take into consideration locations with **no more than two restaurants in radius of 100 meters**, and we want locations **without vegetarian and vegan restaurants in radius of 400 meters**. We will present map of all such locations but also create clusters (using **k-means clustering**) of those locations to identify general zones / neighborhoods / addresses which should be a starting point for final 'street level' exploration and search for optimal venue location by stakeholders.

## Analysis <a name="analysis"></a>

Let's perform some basic explanatory data analysis and derive some additional info from our raw data. First let's count the **number of restaurants in every area candidate**:

In [22]:
location_restaurants_count = [len(res) for res in location_restaurants]

df_locations['Restaurants in area'] = location_restaurants_count

print('Average number of restaurants in every area with radius=300m:', np.array(location_restaurants_count).mean())

df_locations.head(10)

Average number of restaurants in every area with radius=300m: 4.119565217391305


,Address,Latitude,Longitude,X,Y,Distance from center,Restaurants in area
0,"64 scala B, Corso Dante Alighieri, San Salvari...",45.046833,7.674782,-76864.887573,5.014321e+06,2861.817604,6
1,"4/A, Via Carlo Marenco, San Salvario, 10126",45.047320,7.682338,-76264.887573,5.014321e+06,2666.458325,4
2,"6 int. 1, Strada Antica di San Vito, Crimea, C...",45.047807,7.689895,-75664.887573,5.014321e+06,2598.076211,0
3,"19 int. 7, Strada di Revigliasco, Crimea, Circ...",45.048293,7.697452,-75064.887573,5.014321e+06,2666.458325,0
4,"11 int. 5, Strada del Ponte Verde, Crimea, Cir...",45.048778,7.705010,-74464.887573,5.014321e+06,2861.817604,0
5,"131, Corso Re Umberto, Crocetta, 10128",45.050740,7.662851,-77764.887573,5.014841e+06,2954.657341,3
6,"ex Ghia, Via Agostino da Montefeltro, San Seco...",45.051228,7.670408,-77164.887573,5.014841e+06,2563.201124,2
7,"17/I, Corso Raffaello, San Salvario, 10125",45.051716,7.677965,-76564.887573,5.014841e+06,2264.950331,7
8,"Sport Club - Villa Glicini, Viale Diego Balsam...",45.052203,7.685522,-75964.887573,5.014841e+06,2100.000000,2
9,"45, Viale Giacomo Curreno, Crimea, Circoscrizi...",45.052689,7.693080,-75364.887573,5.014841e+06,2100.000000,3


OK, now let's calculate the **distance to nearest vegeterian/vegan restaurant from every area candidate center** (not only those within 300m - we want distance to closest one, regardless of how distant it is).

In [23]:
distances_to_veg_restaurant = []

for area_x, area_y in zip(xs, ys):
    min_distance = 10000
    for res in veg_restaurants.values():
        res_x = res[7]
        res_y = res[8]
        d = calc_xy_distance(area_x, area_y, res_x, res_y)
        if d<min_distance:
            min_distance = d
    distances_to_veg_restaurant.append(min_distance)

df_locations['Distance to veg restaurant'] = distances_to_veg_restaurant

In [24]:
df_locations.head(10)

,Address,Latitude,Longitude,X,Y,Distance from center,Restaurants in area,Distance to veg restaurant
0,"64 scala B, Corso Dante Alighieri, San Salvari...",45.046833,7.674782,-76864.887573,5.014321e+06,2861.817604,6,1555.107229
1,"4/A, Via Carlo Marenco, San Salvario, 10126",45.047320,7.682338,-76264.887573,5.014321e+06,2666.458325,4,1356.572432
2,"6 int. 1, Strada Antica di San Vito, Crimea, C...",45.047807,7.689895,-75664.887573,5.014321e+06,2598.076211,0,1407.913007
3,"19 int. 7, Strada di Revigliasco, Crimea, Circ...",45.048293,7.697452,-75064.887573,5.014321e+06,2666.458325,0,1686.460585
4,"11 int. 5, Strada del Ponte Verde, Crimea, Cir...",45.048778,7.705010,-74464.887573,5.014321e+06,2861.817604,0,2002.235042
5,"131, Corso Re Umberto, Crocetta, 10128",45.050740,7.662851,-77764.887573,5.014841e+06,2954.657341,3,1873.066535
6,"ex Ghia, Via Agostino da Montefeltro, San Seco...",45.051228,7.670408,-77164.887573,5.014841e+06,2563.201124,2,1360.260459
7,"17/I, Corso Raffaello, San Salvario, 10125",45.051716,7.677965,-76564.887573,5.014841e+06,2264.950331,7,955.111923
8,"Sport Club - Villa Glicini, Viale Diego Balsam...",45.052203,7.685522,-75964.887573,5.014841e+06,2100.000000,2,833.168084
9,"45, Viale Giacomo Curreno, Crimea, Circoscrizi...",45.052689,7.693080,-75364.887573,5.014841e+06,2100.000000,3,1093.663260


In [25]:
print('Average distance to closest vegetarian/vegan restaurant from each area center:', df_locations['Distance to veg restaurant'].mean())

Average distance to closest vegetarian/vegan restaurant from each area center: 1238.1253841908585


OK, so **on average a vegan/vegetarian restaurant can be found within ~1.2km** from every area center candidate. That's not close and very good for the stakeholders, but we need to filter our areas carefully!

Let's create a map showing **heatmap / density of restaurants** and try to extract some meaningful info from that. Also, let's show a few circles indicating distance of 1km, 2km and 3km from Piazza Castello.

In [26]:
restaurant_latlons = [[res[2], res[3]] for res in restaurants.values()]

veg_latlons = [[res[2], res[3]] for res in veg_restaurants.values()]

In [27]:
from folium import plugins
from folium.plugins import HeatMap

map_turin = folium.Map(location=lat_lng_coords, zoom_start=13)
folium.TileLayer('cartodbpositron').add_to(map_turin) #cartodbpositron cartodbdark_matter
HeatMap(restaurant_latlons).add_to(map_turin)
folium.Marker(lat_lng_coords).add_to(map_turin)
folium.Circle(lat_lng_coords, radius=1000, fill=False, color='white').add_to(map_turin)
folium.Circle(lat_lng_coords, radius=2000, fill=False, color='white').add_to(map_turin)
folium.Circle(lat_lng_coords, radius=3000, fill=False, color='white').add_to(map_turin)

map_turin

Looks like a few pockets of low restaurant density closest to city center can be found **in the center, south, south-west and mainly north-east from Piazza Castello** (south-east is a luxury residential area not so frequented by the average person).

Let's create another heatmap map showing **heatmap/density of vegetarian/vegan restaurants** only.

In [28]:
map_turin = folium.Map(location=lat_lng_coords, zoom_start=13)
folium.TileLayer('cartodbpositron').add_to(map_turin) #cartodbpositron cartodbdark_matter
HeatMap(veg_latlons).add_to(map_turin)
folium.Marker(lat_lng_coords).add_to(map_turin)
folium.Circle(lat_lng_coords, radius=1000, fill=False, color='white').add_to(map_turin)
folium.Circle(lat_lng_coords, radius=2000, fill=False, color='white').add_to(map_turin)
folium.Circle(lat_lng_coords, radius=3000, fill=False, color='white').add_to(map_turin)

map_turin

This map is not so 'hot' (vegetarian and vegan restaurants represent a subset of only ~2% of all restaurants in Turin) but it also indicates higher density of existing vegan/vegetarian restaurants directly near from Piazza Castello, with closest pockets of **low veg restaurant density positioned north, east and south-west from city center**.

Based on this we will now focus our analysis on areas *north, north-east from Turin center* - we will move the center of our area of interest and reduce it's size to have a radius of **1.5km**. This places our location candidates mostly in boroughs **Borgo Vanchiglia** and **Borgo Dora** (in the map respectively Vanchiglietta and Aurora). Another potentially interesting borough is **Crocetta** with a low restaurant density south/south-west from city center. However this borough is mostly residential, not so popular with tourists and maybe is less interesting to stakeholders.

### Borgo Vanchiglia and Borgo Dora

These are some analysis and reviews I found on the web about these boroughs:

**Borgo Vanchiglia**

"The favorable central position and the proximity to the University of Turin have led, both in Vanchiglia and in Vanchiglietta, to a gradual transformation into residential neighborhoods.
In more recent years, Borgo Vanchiglia has established itself as one of the driving forces of Turin's artistic and commercial creativity. The artisan shops, still present, were joined by various artistic and cultural realities of different fields, such as architecture, design, sculpture, painting, theater, music, cinema. For a few years, the Borgo has become a theater of contrasting realities between the economic crisis that has hit small shops and the new places dedicated to youth nightlife, a question that remains an open topic of discussion today on the nightlife." (wikipedia.org)

**Porta Palazzo (located in Borgo Dora)**

"Open every weekday, it represents the multiethnic heart of Turin. The market area of Porta Palazzo is located close to the Roman Porte Palatine, and has undergone a profound redevelopment in recent years, which however still requires a strong implementation. The ancient art nouveau clock canopy stands out, built in cast iron, iron and glass. It is currently the largest open-air market in Europe with almost 1,000 mobile stalls for the sale of fruit and vegetables, food and clothing." (wikipedia.org)

**Balon (located in Borgo Dora)**

"Balon (pronounced buh-loon) is the biggest flea market in Europe. It takes place each Saturday morning and every second Sunday in Borgo Dora (Aurora), one of the most characteristic boroughs of the area. Here, many antiquity shops put their merchandise on display, making strolling through the market a truly wonderful and often bizarre experience. Expect everything from antique night stands to piles of old TV remotes, from old comic books to used eyeglasses!"(theculturetrip.com)

These boroughs are popular with tourists (one more famous for the artistic life and the nightlife and the other for being the most characteristic boroughs of Turin) relatively close to city center and well connected, those boroughs appear to justify further analysis.

Let's define new, more narrow region of interest, which will include low-restaurant-count parts of Borgo Vanchiglia and Borgo Dora closest to Piazza Castello.

In [29]:
roi_x_min = turin_center_x + 500
roi_y_max = turin_center_y + 400
roi_width = 3000
roi_height = 3000
roi_center_x = roi_x_min + 200
roi_center_y = roi_y_max - 200
roi_center_lon, roi_center_lat = xy_to_lonlat(roi_center_x, roi_center_y)
roi_center = [roi_center_lat, roi_center_lon]

map_turin = folium.Map(location=roi_center, zoom_start=14)
HeatMap(restaurant_latlons).add_to(map_turin)
folium.Marker(lat_lng_coords).add_to(map_turin)
folium.Circle(roi_center, radius=1500, color='white', fill=True, fill_opacity=0.4).add_to(map_turin)

map_turin

Not bad - this nicely covers the big area of low restaurant density in Borgo Vanchiglia and Borgo Dora closest to Turin center.

Let's also create new, more dense grid of location candidates restricted to our new region of interest (let's make our location candidates 100m appart).

In [30]:
k = math.sqrt(3) / 2 # Vertical offset for hexagonal grid cells
x_step = 100
y_step = 100 * k 
roi_y_min = roi_center_y - 400

roi_latitudes = []
roi_longitudes = []
roi_xs = []
roi_ys = []
for i in range(0, int(41/k)):
    y = roi_y_min + i * y_step
    x_offset = 30 if i%2==0 else 0
    for j in range(0, 31):
        x = roi_x_min + j * x_step + x_offset
        d = calc_xy_distance(roi_center_x, roi_center_y, x, y)
        if (d <= 1501):
            lon, lat = xy_to_lonlat(x, y)
            roi_latitudes.append(lat)
            roi_longitudes.append(lon)
            roi_xs.append(x)
            roi_ys.append(y)

print(len(roi_latitudes), 'candidate neighborhood centers generated.')

331 candidate neighborhood centers generated.


OK. Now let's calculate two most important things for each location candidate: **number of restaurants in vicinity** (we'll use radius of **100 meters**) and **distance to closest vegetarian/vegan restaurant**.

In [31]:
def count_restaurants_nearby(x, y, restaurants, radius=100):    
    count = 0
    for res in restaurants.values():
        res_x = res[7]; res_y = res[8]
        d = calc_xy_distance(x, y, res_x, res_y)
        if d<=radius:
            count += 1
    return count

def find_nearest_restaurant(x, y, restaurants):
    d_min = 100000
    for res in restaurants.values():
        res_x = res[7]; res_y = res[8]
        d = calc_xy_distance(x, y, res_x, res_y)
        if d<=d_min:
            d_min = d
    return d_min

roi_restaurant_counts = []
roi_veg_distances = []

print('Generating data on location candidates... ', end='')
for x, y in zip(roi_xs, roi_ys):
    count = count_restaurants_nearby(x, y, restaurants, radius=100)
    roi_restaurant_counts.append(count)
    distance = find_nearest_restaurant(x, y, veg_restaurants)
    roi_veg_distances.append(distance)
print('done.')


Generating data on location candidates... done.


In [32]:
# Let's put this into dataframe
df_roi_locations = pd.DataFrame({'Latitude':roi_latitudes,
                                 'Longitude':roi_longitudes,
                                 'X':roi_xs,
                                 'Y':roi_ys,
                                 'Restaurants nearby':roi_restaurant_counts,
                                 'Distance to veg restaurant':roi_veg_distances})

df_roi_locations.head(10)

,Latitude,Longitude,X,Y,Restaurants nearby,Distance to veg restaurant
0,45.069648,7.69383,-75134.887573,5.016719e+06,4,387.523636
1,45.069729,7.69509,-75034.887573,5.016719e+06,3,422.695076
2,45.069810,7.69635,-74934.887573,5.016719e+06,1,476.621113
3,45.069891,7.69761,-74834.887573,5.016719e+06,0,543.750167
4,45.069972,7.69887,-74734.887573,5.016719e+06,0,619.807069
5,45.070053,7.70013,-74634.887573,5.016719e+06,1,701.895548
6,45.070134,7.70139,-74534.887573,5.016719e+06,2,788.133186
7,45.070215,7.70265,-74434.887573,5.016719e+06,1,872.914504
8,45.070295,7.70391,-74334.887573,5.016719e+06,0,959.611794
9,45.070376,7.70517,-74234.887573,5.016719e+06,0,1048.680056


OK. Let us now **filter** those locations: we're interested only in **locations with no more than two restaurants in radius of 100 meters**, and, most important, **no vegetarian/vegan restaurants in radius of 400m meters**.

In [33]:
good_res_count = np.array((df_roi_locations['Restaurants nearby']<=2))
print('Locations with no more than two restaurants nearby:', good_res_count.sum())

good_veg_distance = np.array(df_roi_locations['Distance to veg restaurant']>=400)
print('Locations with no veg restaurants within 400m:', good_veg_distance.sum())

good_locations = np.logical_and(good_res_count, good_veg_distance)
print('Locations with both conditions met:', good_locations.sum())

df_good_locations = df_roi_locations[good_locations]


Locations with no more than two restaurants nearby: 326
Locations with no veg restaurants within 400m: 304
Locations with both conditions met: 303


Let's see how this looks on a map.

In [34]:
good_latitudes = df_good_locations['Latitude'].values
good_longitudes = df_good_locations['Longitude'].values

good_locations = [[lat, lon] for lat, lon in zip(good_latitudes, good_longitudes)]

map_turin = folium.Map(location=roi_center, zoom_start=14)
folium.TileLayer('cartodbpositron').add_to(map_turin)
HeatMap(restaurant_latlons).add_to(map_turin)
folium.Circle(roi_center, radius=1500, color='white', fill=True, fill_opacity=0.6).add_to(map_turin)
folium.Marker(lat_lng_coords).add_to(map_turin)
for lat, lon in zip(good_latitudes, good_longitudes):
    folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_turin) 

map_turin

Looking good. We now have a bunch of locations mostly in Borgo Vanchiglia, and we know that each of those locations has no more than two restaurants in radius of 100m, and no Italian restaurant closer than 400m. Any of those locations is a potential candidate for a new vegetarian/vegan restaurant, at least based on nearby competition (not considering the south-east residential area that we excluded at the beginning).

Let's now show those good locations in a form of heatmap:

In [35]:
map_turin = folium.Map(location=roi_center, zoom_start=14)
HeatMap(good_locations, radius=20).add_to(map_turin)
folium.Marker(lat_lng_coords).add_to(map_turin)
for lat, lon in zip(good_latitudes, good_longitudes):
    folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_turin)

map_turin

Looking good. What we have now is a clear indication of the zone with low number of restaurants in vicinity, and *no* vegetarian/vegan restaurants at all nearby.

Let us now **cluster** those locations to create **centers of zones containing good locations**. Those zones, their centers and addresses will be the final result of our analysis. 

In [36]:
from sklearn.cluster import KMeans

number_of_clusters = 15

good_xys = df_good_locations[['X', 'Y']].values
kmeans = KMeans(n_clusters=number_of_clusters, random_state=0).fit(good_xys)

cluster_centers = [xy_to_lonlat(cc[0], cc[1]) for cc in kmeans.cluster_centers_]

map_turin = folium.Map(location=roi_center, zoom_start=14)
folium.TileLayer('cartodbpositron').add_to(map_turin)
HeatMap(restaurant_latlons).add_to(map_turin)
folium.Circle(roi_center, radius=1500, color='white', fill=True, fill_opacity=0.4).add_to(map_turin)
folium.Marker(lat_lng_coords).add_to(map_turin)
for lon, lat in cluster_centers:
    folium.Circle([lat, lon], radius=200, color='green', fill=True, fill_opacity=0.25).add_to(map_turin) 
for lat, lon in zip(good_latitudes, good_longitudes):
    folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_turin)

map_turin

Not bad - our clusters represent groupings of most of the candidate locations and cluster centers are placed nicely in our boroughs.

Addresses of those cluster centers will be a good starting point for exploring the neighborhoods to find the best possible location based on neighborhood specifics.

Let's see those zones on a city map without heatmap, using shaded areas to indicate our clusters:

In [37]:
map_turin = folium.Map(location=roi_center, zoom_start=14)
folium.Marker(lat_lng_coords).add_to(map_turin)
for lat, lon in zip(good_latitudes, good_longitudes):
    folium.Circle([lat, lon], radius=100, color='#00000000', fill=True, fill_color='#0066ff', fill_opacity=0.07).add_to(map_turin)
for lat, lon in zip(good_latitudes, good_longitudes):
    folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_turin)
for lon, lat in cluster_centers:
    folium.Circle([lat, lon], radius=200, color='green', fill=False).add_to(map_turin) 

map_turin

Let's zoom in on candidate areas in **Borgo Vanchiglia**:

In [38]:
# Corso Tortona is an important street of Borgo Vanchiglia
lat_lng_BV = None

while(lat_lng_BV is None):
  g = geocoder.osm('{}, Corso Tortona, Torino, Italia')
  lat_lng_BV = g.latlng

In [39]:
map_turin = folium.Map(location=lat_lng_BV, zoom_start=15)
folium.Marker(lat_lng_coords).add_to(map_turin)
for lon, lat in cluster_centers:
    folium.Circle([lat, lon], radius=200, color='green', fill=False).add_to(map_turin) 
for lat, lon in zip(good_latitudes, good_longitudes):
    folium.Circle([lat, lon], radius=100, color='#0000ff00', fill=True, fill_color='#0066ff', fill_opacity=0.07).add_to(map_turin)
for lat, lon in zip(good_latitudes, good_longitudes):
    folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_turin)

map_turin

...and candidate areas in **Borgo Dora**:

In [40]:
# via Bologna is an important street of Borgo Dora
lat_lng_BD = None

while(lat_lng_BD is None):
  g = geocoder.osm('{}, via Bologna, Torino, Italia')
  lat_lng_BD = g.latlng

In [41]:
map_turin = folium.Map(location=lat_lng_BD, zoom_start=15)
folium.Marker(lat_lng_coords).add_to(map_turin)
for lon, lat in cluster_centers:
    folium.Circle([lat, lon], radius=200, color='green', fill=False).add_to(map_turin) 
for lat, lon in zip(good_latitudes, good_longitudes):
    folium.Circle([lat, lon], radius=100, color='#0000ff00', fill=True, fill_color='#0066ff', fill_opacity=0.07).add_to(map_turin)
for lat, lon in zip(good_latitudes, good_longitudes):
    folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_turin)

map_turin

Finaly, let's **reverse geocode those candidate area centers to get the addresses** which can be presented to stakeholders.

In [42]:
candidate_area_addresses = []
print('==============================================================')
print('Addresses of centers of areas recommended for further analysis')
print('==============================================================\n')
for lon, lat in cluster_centers:
    addr = get_address(lat, lon).replace(', Italia', '')
    addr = addr.replace(', Italy', '')
    addr = addr.replace(', Torino, Provincia di Torino, Piemonte', '')
    candidate_area_addresses.append(addr)    
    x, y = lonlat_to_xy(lon, lat)
    d = calc_xy_distance(x, y, turin_center_x, turin_center_y)
    print('{}{} => {:.1f}km from Piazza Castello'.format(addr, ' '*(50-len(addr)), d/1000))
    

Addresses of centers of areas recommended for further analysis

9/B, Via Cesare Balbo, Vanchiglia, 10124           => 1.0km from Piazza Castello
Rossini, Circoscrizione 7, 10153                   => 2.0km from Piazza Castello
50/E, Via Bologna, Rossini, Aurora, 10152          => 1.3km from Piazza Castello
35/A, Via Lessolo, Vanchiglietta, 10153            => 1.9km from Piazza Castello
Università degli Studi di Torino, Via Modena, Rossini, Aurora, 10124 => 1.1km from Piazza Castello
71/A, Corso Regio Parco, Rossini, Aurora, 10154    => 2.0km from Piazza Castello
29/A, Via Catania, Rossini, Aurora, 10153          => 1.1km from Piazza Castello
113, Corso Novara, Monte Bianco, Barriera di Milano, 10154 => 1.5km from Piazza Castello
14/A, Corso Belgio, Vanchiglia, 10153              => 1.4km from Piazza Castello
26, Via Modena, Rossini, Aurora, 10152             => 1.0km from Piazza Castello
Rossini, Circoscrizione 7                          => 1.5km from Piazza Castello
34 int. 7, Via Poli

This concludes our analysis. We have created 15 addresses representing centers of zones containing locations with low number of restaurants and no vegetarian/vegan restaurants nearby, all zones being fairly close to city center (all less than 2.5km from Piazza Castello, and six of those less than 1.5km from Piazza Castello). Although zones are shown on map with a radius of ~100 meters (green circles), their shape is actually very irregular and their centers/addresses should be considered only as a starting point for exploring area neighborhoods in search for potential restaurant locations. Most of the zones are located in Borgo Vanchiglia and Borgo Dora boroughs, which we have identified as interesting due to being popular with tourists, fairly close to city center and well connected by public transport.

In [43]:
map_turin = folium.Map(location=roi_center, zoom_start=14)
folium.Circle(lat_lng_coords, radius=50, color='red', fill=True, fill_color='red', fill_opacity=1).add_to(map_turin)
for lonlat, addr in zip(cluster_centers, candidate_area_addresses):
    folium.Marker([lonlat[1], lonlat[0]], popup=addr).add_to(map_turin) 
for lat, lon in zip(good_latitudes, good_longitudes):
    folium.Circle([lat, lon], radius=250, color='#0000ff00', fill=True, fill_color='#0066ff', fill_opacity=0.05).add_to(map_turin)
map_turin

## Results and Discussion <a name="results"></a>

Our analysis shows that although there is a great number of restaurants in Turin (420 in our initial area of interest which was 3x3km around Piazza Castello), there are pockets of low restaurant density in Borgo Dora, in Borgo Vanchiglia and in Crocetta boroughs. Highest concentration of restaurants was detected around Piazza Castello, so we focused our attention to areas north, north-east, corresponding to boroughs Borgo Dora and Borgo Vanchiglia borough. Another borough was identified as potentially interesting (Crocetta, south-west from Piazza Castello), but our attention was focused on Borgo Dora and Borgo Vanchiglia which offer a combination of popularity among tourists, closeness to city center and a number of pockets of low restaurant density.

After directing our attention to this more narrow area of interest (covering approx. 2x2km north-east from Piazza Castello) we first created a dense grid of location candidates (spaced 100m appart); those locations were then filtered so that those with more than two restaurants in radius of 100m and those with a veg restaurant closer than 400m were removed.

Those location candidates were then clustered to create zones of interest which contain greatest number of location candidates. Addresses of centers of those zones were also generated using reverse geocoding to be used as markers/starting points for more detailed local analysis based on other factors.

Result of all this is 15 zones containing largest number of potential new restaurant locations based on number of and distance to existing venues - both restaurants in general and veg restaurants particularly. This, of course, does not imply that those zones are actually optimal locations for a new restaurant! Purpose of this analysis was to only provide info on areas close to Turin center but not crowded with existing restaurants (particularly vegetarian/vegan) - it is entirely possible that there is a very good reason for small number of restaurants in any of those areas, reasons which would make them unsuitable for a new restaurant regardless of lack of competition in the area. Recommended zones should therefore be considered only as a starting point for more detailed analysis which could eventually result in location which has not only no nearby competition but also other factors taken into account and all other relevant conditions met.

## Conclusion <a name="conclusion"></a>

Purpose of this project was to identify Turin areas close to center with low number of restaurants (particularly veg restaurants) in order to aid stakeholders in narrowing down the search for optimal location for a new vegetarian/vegan restaurant. By calculating restaurant density distribution from Foursquare data we have first identified general boroughs that justify further analysis (Borgo Dora and Borgo Vanchiglia), and then generated extensive collection of locations which satisfy some basic requirements regarding existing nearby restaurants. Clustering of those locations was then performed in order to create major zones of interest (containing greatest number of potential locations) and addresses of those zone centers were created to be used as starting points for final exploration by stakeholders.

Final decision on optimal restaurant location will be made by stakeholders based on specific characteristics of neighborhoods and locations in every recommended zone, taking into consideration additional factors like attractiveness of each location (proximity to park or water), levels of noise / proximity to major roads, real estate availability, prices, social and economic dynamics of every neighborhood etc.